In [ ]:
import sys
sys.path.append("..")
#import matplotlib
#%matplotlib inline
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

import numpy as np
from LowLevelModules.ImportSDT import SdtFile
from LowLevelModules.GeneralFunctions import get_work_dir, prettify_plot

# fitting the data
# def g2(t, I0, t0, t1, t2):
#     return I0*(t2-np.exp(-abs(t-t0)/t1))/t2

def g2(t, t0, tau, N):
    return (N-1)/N + 1/N*(1-np.exp(-abs(t-t0)/tau))

file = r"Z:\Projects\WSe2\2019\12 december\2019-12-06 hBN WSe2 300nm pillars Tom cooldown\g2\pillar4c5_5K_500nW_5V"
sdt = SdtFile(file + '.sdt')
delay = -40.71
times = sdt.times[0]
info = sdt.measure_info[0]
x = times*1e9 + delay
y = sdt.data[0][0]
dt_bin= (times[-1]-times[0])/(len(times)-1)
col_t = info['col_t'][0]

#trim data
x = x[y>0]
y = y[y>0]

#ct_rate1
info_ctr = info['StopInfo']
min_ctr11 = info_ctr['min_sync_rate'][0]
min_ctr12 = info_ctr['min_cfd_rate'][0]
min_ctr13 = info_ctr['min_tac_rate'][0]
max_ctr11 = info_ctr['max_sync_rate'][0]
max_ctr12 = info_ctr['max_cfd_rate'][0]
max_ctr13 = info_ctr['max_tac_rate'][0]

Ct_rate_APD1 = (min_ctr12 + max_ctr12) / 2
Ct_rate_APD2 = (min_ctr11 + max_ctr11) / 2
Ct_rate_APD1 = 14*1000
Ct_rate_APD2 = 16*1000
print(Ct_rate_APD1, Ct_rate_APD2, col_t, dt_bin)
#normalization
norm = float(Ct_rate_APD1 * Ct_rate_APD2 * dt_bin * col_t)
#norm = float(max_ctr11 * max_ctr12 * dt_bin * col_t)

y = np.divide(y, np.mean(y[656:]), casting="unsafe")
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.plot(x, y, 'ko', markersize=5)
ax.plot([x[0], x[-1]], [0.5, 0.5], 'k--')

#do the fit
coeffs, pcov = curve_fit(g2, x, y)
perr = np.sqrt(np.diag(pcov))
g20 = 1-1/coeffs[2]
perr_g2 = g20 * perr[2] / coeffs[2]
y_arrf = g2(x, *coeffs)
ax.plot(x, y_arrf, 'b-', linewidth=2)

prettify_plot(ax, "Time (ns)", "Counts (a.u.)")
ax.set_xlim([-32, 40])
ax.set_ylim([0, np.max(y)])
ax.text(8, 0.1, f"g2(0) = {np.round(g20, 3)} ± {np.round(perr_g2, 3)}", fontsize=18)
fig.savefig(file + '.png', format='png', bbox_inches='tight')
print(f"g2(0) = {np.round(g20, 3)} ± {np.round(perr_g2, 3)}")